In [1]:
from glob import glob
from pathlib import Path

import pandas as pd
import re

## LOAD PATTERNS

In [2]:
df_patterns = pd.read_excel(Path().absolute().parent.parent/ "data/patterns.xlsx")

In [3]:
dic_patterns = {d["Nome_variável"]:r"{}".format(d["Expressão regular"]) for d in df_patterns.to_dict("records")}

In [4]:
dic_patterns

{'Consumidor Telecom Telas': 'ausência de prova.{,200}(telas?|registros?)\\s*(sist[êe]mic[ao]|de sistema)',
 'CondESC': 'Cc]onden[o|a].{2,30}distrito federal',
 'Indenizatória Improcedente': 'consumidor.*inscrição devida',
 'Processos judiciais': '\\s\\d{7}\\-\\d{2}\\.\\d{4}\\.\\d{1}\\.\\d{2}\\.\\d{4}',
 'Achar Lei': 'Lei\\s+[\\d\\.]{,6}/\\w{2,4}'}

## FILES TO EXTRACT REGULAR EXPRESSIONS FROM

In [5]:
PATH_FILES = str(Path().absolute().parent.parent/ "data/regular_expressions_class")
files = glob(PATH_FILES + "/**/*.xlsx", recursive=True)

## CREATING A BOOLEAN REPORT OF THE TEXTS

In [6]:
COL_ID = "id"
COL_TEXT = "text"
NAME_REPORT = str(Path().absolute().parent.parent/ "reports/extraction_class.xlsx")

In [7]:
dic_results = {}
for f in files:
    df_f = pd.read_excel(f)
    for row in df_f.to_dict("records"):
        if row[COL_ID] not in dic_results:
            dic_results[row[COL_ID]] = {k:0 for k in dic_patterns}
        for name, exp in dic_patterns.items():
            try:
                if re.search(exp, str(row[COL_TEXT]), flags=re.I|re.S):
                    dic_results[row[COL_ID]][name] += 1
            except:
                print(exp)
                print(row[COL_TEXT])

In [8]:
rows = []
for id_lawsuit, res in dic_results.items():
    dic_aux = {COL_ID:id_lawsuit}
    dic_aux.update(res)
    rows.append(dic_aux)
df_report = pd.DataFrame(rows)
df_report.to_excel(NAME_REPORT, index=False)